# Evaluating Agents

上一节中我们使用LangGraph创建了第一个agent，可以帮我们回复邮件（真正回复的功能还没有实现），再继续实现更多功能之前，我们先来学习一下如何评估（evaluation）agent的效果。
![overview-img](img/overview_eval.png)

测试环节对于一个agent能否部署到生成环境有着非常重要的影响，只有通过测试，才能知道诸如回复质量、Token消耗量、延迟、分类准确性等一些列量化的指标。

我们将使用 [LangSmith](https://docs.smith.langchain.com/) 来完成这个任务，LangSmith 是 LangChain 团队开发的，用于评估和监控 LLM 应用的平台，它提供了两种主要的方式来测试agent。

Load Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")
import os
import sys
sys.path.append("..")

## How to run Evaluations

### Pytest / Vitest

Pytest 和 Vitest 分别是 Python 和 JavaScript 的单元测试框架，用于运行测试代码并生成测试报告，LangSmith 中集成了这些框架，可以方便的将测试报告同步到 LangSmith 中。在本项目中，我们使用 Pytest 来运行测试代码。

Pytest 语法简洁，很容易上手，而且，对于那些有复杂逻辑，很难通用形式处理的测试场景，Pytest 也提供了足够的灵活性来处理。

### LangSmith Datasets

除了开发语言本身所提供的单元测试框架外，LangSmith 还提供了自己的数据集（Datasets）管理功能，你可以通过 LangSmith 的 API 来使用这些数据集测试你的应用。
* LangSmith datasets are great for teams who are collaboratively building out their test suite. 
* You can leverage production traces, annotation queues, synthetic data generation, and more, to add examples to an ever-growing golden dataset.
* LangSmith datasets are great when you can define evaluators that can be applied to every test case in the dataset (ex. similarity, exact match accuracy, etc.)

> Note: _annotation queues_ 是一种数据标注系统使用的队列，作用是将需要标注的数据有序的分配给标注人员，并跟踪管理标注进度、质量和一致性。


## 测试用例（Test Cases）

测试永远都是从定义测试用例开始的，这一步很重要也很有挑战性，因为你必须清楚的知道你的agent要做什么，你才能定义出正确的测试用例。

这里事先定义好了一些测试用例，每个用例包含以下字段：
- `email_input`: 输入的邮件内容
- `expected_tool_calls`: 期待的工具调用结果
- `triage_output`: 期待的分类结果
- `response_criteria`: 回复的邮件内容应该满足的标准

In [11]:
%load_ext autoreload
%autoreload 2

# eval.email_dataset 中定义好了一组测试用例
from src.eval.email_dataset import email_inputs, expected_tool_calls, triage_outputs_list, response_criteria_list

test_case_ix = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


下面我们逐个来查看一下这些测试用例的内容。

In [ ]:
print("Email Input:", email_inputs[test_case_ix])

Email Input: <class 'dict'> {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lance Martin <lance@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi Lance,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}


Email Input 中包含 author、to、subject、email_thread 等信息，agent的作用就是要根据这些信息来决定该如何回复邮件。

In [10]:
print("Expected Tool Calls:", expected_tool_calls[test_case_ix])

Expected Tool Calls: ['write_email', 'done']


Expected Tool Calls 中存储了agent在执行过程中依次需要执行的工具名称，在上面这个用例中，agent需要依次调用 `write_email` 和 `done` 两个工具。

In [13]:
print("Triage Output:", triage_outputs_list[test_case_ix])

Triage Output: respond


Triage Output 是用来评估分类节点效果的，对应的三种可能值分别是 ignore, notify, respond。

In [14]:
print("Response Criteria:", response_criteria_list[test_case_ix])

Response Criteria: 
• Send email with write_email tool call to acknowledge the question and confirm it will be investigated  



Response Criteria 是针对需要回复的邮件，agent所撰写的邮件应该满足什么样的要求。

**通过上面4个测试标准可以看出，我们的测试既包含端到端的结果测试，也包含特定步骤的中间过程测试。**

## Pytest 测试

基于上面介绍的内容，首先我们来使用 Pytest 测试一下agent工具调用的效果怎么样。

In [ ]:
import pytest
from src.eval.email_dataset import email_inputs, expected_tool_calls
from src.utils import format_messages_string
from src.utils import extract_tool_calls
from src.email_assistant import email_assistant

from langsmith import testing as t

`email_assistant` 是我们上节内容构建的agent，也是我们要测试的对象。

In [23]:
@pytest.mark.langsmith
@pytest.mark.parametrize(
    "email_input, expected_tool_calls",
    [
        (email_inputs[0], expected_tool_calls[0]),
        (email_inputs[3], expected_tool_calls[3]),
    ]   
)
def test_email_dataset_tool_calls(email_input, expected_tool_calls):
    """Test if email processing contains expected tool calls.

    这里只测试了工具是否被正确调用，没有测试调用顺序。
    """

    # Run the email assistant
    result = email_assistant.invoke({"email_input": email_input})

    # Extract tool calls from the result
    extracted_tool_calls = extract_tool_calls(result["messages"])

    # check if all expected tool calls are in the extracted ones
    missing_calls = [call for call in expected_tool_calls if call not in extracted_tool_calls]

    t.log_outputs({
        "missing_calls": missing_calls,
        "extracted_tool_calls": extracted_tool_calls,
        "response": format_messages_string(result["messages"])
    })

    assert len(missing_calls) == 0


上述代码完整的展示了如何使用 Pytest 和 LangSmith 来测试一个智能体。有几点内容需要注意一下：
- 只需要添加装饰器 `@pytest.mark.langsmith`，就可以将测试结果自动上传到 LangSmith 进行查看；
- 通过装饰器 `@pytest.mark.parameterize`，可以将测试用例参数化。

### Running Pytest

我们需要再命令行中运行上述代码，首先我们需要将代码整理到 `test_tools.py` 文件中，然后在命令行中执行：
```bash
LANGSMITH_TEST_SUITE='Email assistant 04: Test Tools For Interrupt'  pytest test_tools.py
```

### 查看结果

运行结束之后，在 LangSmith 平台（https://smith.langchain.com/） 的 Datasets & Experiments 页面可以看到你的测试结果。

![langsmith_pytest.png](img/langsmith_pytest.png)

## LangSmith Datasets 测试

学习完了使用 Pytest 测试工具调用的结果之后，接着学习如何使用 LangSmith Datasets 来测试邮件分类（triage_router）的效果。

下图展示了 LangSmith Datasets 的工作流程，Dataset Examples 中的 inputs 会作为参数传入到 Agent 中（也就是`eamil_assistant`），然后将 Agent  的输出和 reference outputs 通过 Test Function 进行比较，最后输入测试结果。

![overview-img](img/eval_detail.png)

### 定义 Dataset